In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms.functional as F
import random
from PIL import ImageChops
import math
import sys

### Loading Configuration

In the following steps, we will load the configuration settings using the `load_configuration` function. The configuration is stored in the `config` variable which will b

In [2]:


print("CWD:", os.getcwd())
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from config.load_configuration import load_configuration
config = load_configuration()
#os.chdir(original_cwd)  # Restore original CWD

CWD: c:\Users\lobster\Documents\HKA\Kurse\VDKI\Projekt\GIT_VDKI\HKA_VDKI\preprocessing
PC Name: DESKTOP-BLKLC13
Loaded configuration from ../config/config_luka.yaml


Seeds oder ähnliches Setzten, um wiederholbarkeit herzustellen??
Bei den Splits habe ich seeds händisch gesetzt

In [8]:
# Paths
source_dir = config['path_to_raw_pics']
output_dir = config['path_to_split_aug_pics']
#source_dir =  "C:\\Users\\lobster\\Documents\\HKA\\Kurse\\VDKI\\Projekt\\test_photos"
#output_dir =  "C:\\Users\\lobster\\Documents\\HKA\\Kurse\\VDKI\\Projekt\\test_data"
class_names = os.listdir(source_dir)  # e.g., ['bunny', 'cat', 'dog']

print(source_dir)

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Make folders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Split and move files (no subfolders)
for class_name in class_names:
    class_dir = os.path.join(source_dir, class_name)
    files = os.listdir(class_dir)

    train_val, test = train_test_split(files, test_size=test_ratio, random_state=config['seed'])
    train, val = train_test_split(train_val, test_size=val_ratio / (train_ratio + val_ratio), random_state=config['seed'])

    for split, split_files in zip(['train', 'val', 'test'], [train, val, test]):
        for file in split_files:
            src = os.path.join(class_dir, file)

            # Prefix with class name to prevent collisions
            filename = f"{file}"
            dst = os.path.join(output_dir, split, filename)

            shutil.copy2(src, dst)

C:\Users\lobster\SynologyDrive\SS25_MSYS_KAER-AI-PoseAct\21_Test_Data\data_scraper\Google-Image-Scraper\Bilder_Marie


In [4]:
print(class_names)

['Kleintiere', 'Zwergkaninchen']


### Augmentation

In [9]:
# Define transforms
flip = T.RandomHorizontalFlip(p=1.0)

def rotate_crop_borders(img, angle):
    rotate_plus = T.RandomRotation(degrees=(angle, angle))
    img = rotate_plus(img)

    angle_deg = angle % 180
    if angle_deg > 90:
        angle_deg = 180 - angle_deg
    angle_rad = math.radians(angle_deg)

    sin_a = math.sin(angle_rad)
    cos_a = math.cos(angle_rad)

    if img.height * sin_a <= img.width * cos_a:
        new_w = (img.width * cos_a) - (img.height * sin_a)
        new_h = (img.height * cos_a) - (img.width * sin_a)
    else:
        new_w = (img.height * cos_a) - (img.width * sin_a)
        new_h = (img.width * cos_a) - (img.height * sin_a)

    # Skip invalid sizes
    if new_w <= 0 or new_h <= 0:
        print(f"Skipping rotation {angle}°: invalid crop size (w={new_w:.2f}, h={new_h:.2f})")
        return None

    left = (img.width - new_w) // 2
    top = (img.height - new_h) // 2
    right = left + new_w
    bottom = top + new_h

    if right <= left or bottom <= top:
        print(f"Skipping rotation {angle}°: crop box is invalid (left={left}, top={top}, right={right}, bottom={bottom})")
        return None

    try:
        cropped_img = img.crop((left, top, right, bottom))
    except Exception as e:
        print(f"Crop failed at angle {angle}°: {e}")
        return None

    # Skip if aspect ratio is too extreme
    w, h = cropped_img.size
    if max(w, h) > 4 * min(w, h):
        print(f"Skipping rotation {angle}°: extreme aspect ratio ({w}x{h})")
        return None

    return cropped_img


def adjust_gamma(image, gamma):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    inv_gamma = 1.0 / gamma
    table = [((i / 255.0) ** inv_gamma) * 255 for i in range(256)]
    table = np.array(table).astype("uint8")
    r, g, b = image.split()
    r = r.point(table)
    g = g.point(table)
    b = b.point(table)
    return Image.merge('RGB', (r, g, b))

def apply_extra_augments(img):
    # Randomize jitter parameters
    brightness = random.uniform(0, 0.3)
    contrast = random.uniform(0, 0.3)
    hue = random.uniform(0, 0.1)
    saturation = random.uniform(0, 0.3)

    # Apply jitter and blur
    jitter = T.ColorJitter(brightness=brightness, contrast=contrast)
    hue_sat = T.ColorJitter(hue=hue, saturation=saturation)

    img = jitter(img)
    img = hue_sat(img)

    if random.random() < 0.7:  # Optional: Blur sometimes
        img = T.GaussianBlur(kernel_size=5)(img)
    if random.random() < 0.7:  # Optional: Blur sometimes
        gamma = random.uniform(0.4, 0.8)
        img = adjust_gamma(img, gamma)

    return img

# Valid image extensions
valid_exts = ('.jpg', '.jpeg', '.png')

# Loop through class folders
for class_name in os.listdir(output_dir):
    class_path = os.path.join(output_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    for filename in os.listdir(class_path):
        brightness = random.uniform(0, 0.4)
        contrast = random.uniform(0, 0.4)
        hue = random.uniform(0, 0.2)
        saturation = random.uniform(0, 0.4)
        anglePlus = random.uniform(5, 15)
        angleMinus = random.uniform(-5, -15)
        
        if not filename.lower().endswith(valid_exts):
            continue

        filepath = os.path.join(class_path, filename)
        name, ext = os.path.splitext(filename)

        # Load original image
        original = Image.open(filepath).convert("RGB")

        # Detect and preserve original label suffix (_ok or _nok)
        if '_ok' in name:
            base_name = name.split('_ok')[0] + '_ok'
        elif '_nok' in name:
            base_name = name.split('_nok')[0] + '_nok'
        else:
            base_name = name  # fallback if no label is found

        # v0 - original, saved as _v0
        v0_path = os.path.join(class_path, f"{base_name}_v0{ext}")
        original.save(v0_path)

        # Optionally remove the original if it’s not already a _v0 version
        if not name.endswith('_v0'):
            try:
                os.remove(filepath)
                print(f"Deleted original: {filename}")
            except Exception as e:
                print(f"Could not delete original {filename}: {e}")


        # v1 - flipped
        v1 = flip(original)
        v1.save(os.path.join(class_path, f"{base_name}_v1{ext}"))

         # v2 - rotated +5° + jitter + blur
        v2 = rotate_crop_borders(original, anglePlus)
        if v2 is not None:
            v2 = apply_extra_augments(v2)
            v2.save(os.path.join(class_path, f"{base_name}_v2{ext}"))

        # v3 - rotated -5° + jitter + blur
        v3 = rotate_crop_borders(original, angleMinus)
        if v3 is not None:
            v3 = apply_extra_augments(v3)
            v3.save(os.path.join(class_path, f"{base_name}_v3{ext}"))

        # v4 - flipped + rotated +5° + jitter + blur
        v4 = rotate_crop_borders(original, anglePlus)
        if v4 is not None:
            v4 = flip(v4)
            v4 = apply_extra_augments(v4)
            v4.save(os.path.join(class_path, f"{base_name}_v4{ext}"))

        # v5 - flipped + rotated -5° + jitter + blur
        v5 = rotate_crop_borders(original, angleMinus)
        if v5 is not None:
            v5 = flip(v5)
            v5 = apply_extra_augments(v5)
            v5.save(os.path.join(class_path, f"{base_name}_v5{ext}"))





        #print(f"Generated 6 variants for: {filename}")

Deleted original: Eichhoernchen_0014_nok.jpg
Deleted original: Eichhoernchen_0030_nok.jpg
Deleted original: Eichhoernchen_0045_nok.jpg
Deleted original: Eichhoernchen_0052_nok.jpg
Deleted original: Eichhoernchen_0057_nok.jpg
Deleted original: Eichhoernchen_0066_nok.jpg
Deleted original: Eichhoernchen_0068_nok.jpg
Deleted original: Eichhoernchen_0070_nok.jpg
Deleted original: Eichhoernchen_0071_nok.jpg
Deleted original: Eichhoernchen_0097_nok.jpg
Deleted original: Eichhoernchen_0100_nok.jpg
Deleted original: Eichhoernchen_0101_nok.jpg
Deleted original: Eichhoernchen_0106_nok.jpg
Deleted original: Eichhoernchen_0110_nok.jpg
Deleted original: Eichhoernchen_0112_nok.jpg
Deleted original: Eichhoernchen_0121_nok.jpg
Deleted original: Eichhoernchen_0122_nok.jpg
Deleted original: Eichhoernchen_0125_nok.jpg
Deleted original: Eichhoernchen_0129_nok.jpg
Deleted original: Eichhoernchen_0135_nok.jpg
Deleted original: Eichhoernchen_0136_nok.jpg
Deleted original: Eichhoernchen_0169_nok.jpg
Deleted or

c:\Users\lobster\anaconda3\envs\VDKI-Projekt\Lib\site-packages\PIL\Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Deleted original: Kaninchen_1061_ok.png
Deleted original: Kaninchen_1062_ok.jpeg
Deleted original: Kaninchen_1063_ok.jpeg
Deleted original: Kaninchen_1065_ok.jpeg
Deleted original: Kaninchen_1066_ok.jpeg
Deleted original: Kaninchen_1067_ok.jpeg
Deleted original: Kaninchen_1068_ok.jpeg
Deleted original: Kaninchen_1069_ok.jpeg
Deleted original: Kaninchen_1070_ok.png
Deleted original: Kaninchen_1072_ok.jpeg
Deleted original: Kaninchen_1074_ok.jpeg
Deleted original: Kaninchen_1075_ok.jpeg
Deleted original: Kaninchen_1077_ok.jpeg
Deleted original: Kaninchen_1079_ok.jpeg
Deleted original: Kaninchen_1080_ok.jpeg
Deleted original: Kaninchen_1081_ok.jpeg
Deleted original: Katze_0001_nok.jpg
Deleted original: Katze_0003_nok.jpg
Deleted original: Katze_0005_nok.jpg
Deleted original: Katze_0006_nok.jpg
Deleted original: Katze_0009_nok.jpg
Deleted original: Katze_0011_nok.jpg
Deleted original: Katze_0013_nok.jpg
Deleted original: Katze_0014_nok.jpg
Deleted original: Katze_0015_nok.jpg
Deleted origi

c:\Users\lobster\anaconda3\envs\VDKI-Projekt\Lib\site-packages\PIL\TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


Deleted original: Meerschweinchen_0062_nok.jpg
Deleted original: Meerschweinchen_0063_nok.jpg
Deleted original: Meerschweinchen_0064_nok.jpg
Deleted original: Meerschweinchen_0065_nok.jpg
Deleted original: Meerschweinchen_0066_nok.jpg
Deleted original: Meerschweinchen_0067_nok.jpg
Deleted original: Meerschweinchen_0068_nok.jpg
Deleted original: Meerschweinchen_0070_nok.jpg
Deleted original: Meerschweinchen_0071_nok.jpg
Deleted original: Meerschweinchen_0072_nok.jpg
Deleted original: Meerschweinchen_0073_nok.jpg
Deleted original: Meerschweinchen_0074_nok.jpg
Deleted original: Meerschweinchen_0077_nok.jpg
Deleted original: Meerschweinchen_0079_nok.jpg
Deleted original: Meerschweinchen_0080_nok.jpg
Deleted original: Meerschweinchen_0081_nok.jpg
Deleted original: Meerschweinchen_0082_nok.jpg
Deleted original: Meerschweinchen_0086_nok.jpg
Deleted original: Meerschweinchen_0088_nok.jpg
Deleted original: Meerschweinchen_0089_nok.jpg
Deleted original: Meerschweinchen_0091_nok.jpg
Deleted origi